In [1]:
import os

In [2]:
%pwd

'd:\\Courses\\Udemy\\Complete MLOps Bootcamp\\Projects\\7_End to End Projects\\end-to-end-data-science-project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Courses\\Udemy\\Complete MLOps Bootcamp\\Projects\\7_End to End Projects\\end-to-end-data-science-project'

In [9]:
import pandas as pd

data = pd.read_csv('artifacts/data_ingestion/WineQT.csv')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
 12  Id                    1143 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 116.2 KB


In [9]:
data.isnull().sum()

Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0


In [11]:
data.shape

(1143, 13)

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [16]:
from src.project.constants import *
from src.project.utils.common import read_yaml, create_directories
from src.project import logger
import zipfile

In [17]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                  params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)   

        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self)-> DataValidationConfig:
        config=self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])

        data_validation_config= DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema)
        
        return data_validation_config

In [18]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self)-> bool:
        try:
            validation_status = True
            errors = []
          
            data = pd.read_csv(self.config.unzip_data_dir)
            
            # Get actual columns and their dtypes
            all_cols = dict(data.dtypes)  # {col_name: dtype}

            # Get schema columns and dtypes
            all_schema = self.config.all_schema  # {col_name: dtype}

            
            for col, dtype in all_cols.items():
                if col not in all_schema:
                    validation_status = False
                    errors.append(f"Column missing in schema: {col}")
                elif str(dtype) != str(all_schema[col]):
                    validation_status = False
                    errors.append(f"Dtype mismatch for '{col}': expected {all_schema[col]}, got {dtype}")

            # Also check if any schema columns are missing in the dataset
            for col in all_schema:
                if col not in all_cols:
                    validation_status = False
                    errors.append(f"Column missing in dataset: {col}")

            # Save validation status
            with open(self.config.STATUS_FILE, 'w') as f:
                if validation_status:
                    f.write(f"Data Validation found no errors!\n")
                    f.write(f"Validation Status: {validation_status}")
                if errors:
                    f.write('Data Validation found errors! Check STATUS_FILE for details.\n')
                    f.write("Errors:\n")
                    for err in errors:
                        f.write(f"{err}\n")
                    f.write(f"Validation Status: {validation_status}")
                        
            
            return validation_status

        except Exception as e:
            raise e

In [19]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-09-07 00:30:39,327: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-07 00:30:39,338: INFO: common: yaml file: params.yaml loaded successfully]


[2025-09-07 00:30:39,379: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-07 00:30:39,388: INFO: common: created directory at: artifacts]
[2025-09-07 00:30:39,390: INFO: common: created directory at: artifacts/data_validation]


In [13]:
with open(Path('artifacts/data_validation/status.txt'), 'r') as f:
                status = f.read().split()[-1]
                print(type(status))
                

<class 'str'>
